# 🎬 Dataset Sintético - Churn de Streaming Brasileiro

## Hackathon - Análise e Predição de Churn

---


### 🎯 Objetivo

Este notebook gera um dataset sintético que emula com  fidelidade o comportamento de dados reais de uma plataforma de streaming, incluindo:

- Correlações realistas entre variáveis
- Distribuições estatísticas que refletem padrões do mercado
- Regras de negócio consistentes
- Missing values com padrões realistas, explicativos Genero e avaliações.
- Taxa de churn de ~25% (desbalanceado, como no mundo real)

---

## 1. Importação de Bibliotecas e Configurações Iniciais

In [1]:
# Bibliotecas necessárias
import numpy as np
import pandas as pd
from scipy import stats
from scipy.special import expit 
from faker import Faker
import warnings
warnings.filterwarnings('ignore')

# Seed para reprodutibilidade
np.random.seed(42)

# Configurações principais
N_REGISTROS = 30000
CHURN_RATE_TARGET = 0.25

print("=" * 60)
print("CONFIGURAÇÃO DO DATASET SINTÉTICO")
print("=" * 60)
print(f"\n📊 Parâmetros:")
print(f"   • Total de registros: {N_REGISTROS:,}")
print(f"   • Taxa de churn target: {CHURN_RATE_TARGET:.1%}")
print(f"   • Churners esperados: ~{int(N_REGISTROS * CHURN_RATE_TARGET):,}")
print(f"   • Ativos esperados: ~{int(N_REGISTROS * (1-CHURN_RATE_TARGET)):,}")

CONFIGURAÇÃO DO DATASET SINTÉTICO

📊 Parâmetros:
   • Total de registros: 31,351
   • Taxa de churn target: 26.0%
   • Churners esperados: ~8,151
   • Ativos esperados: ~23,199


## 2. Definição de Parâmetros do Negócio

Os parâmetros abaixo foram definidos com base em benchmarks do mercado de streaming brasileiro e padrões de comportamento de consumidores digitais.

In [2]:
# =============================================================================
# PARÂMETROS DE NEGÓCIO
# =============================================================================

# Distribuição regional baseada em penetração de streaming no Brasil
# Fonte: Dados aproximados de penetração de internet e streaming por região
REGIOES = {
    'Sudeste': 0.42,      # Maior concentração populacional e renda
    'Nordeste': 0.27,     # Segunda maior população
    'Sul': 0.15,          # Alta penetração de internet
    'Centro-Oeste': 0.09, # Menor população
    'Norte': 0.07         # Menor penetração digital
}

# Gêneros com probabilidades (incluindo missing - campo optativo)
GENEROS = {
    'Masculino': 0.42,
    'Feminino': 0.40,
    'Prefiro Não Informar': 0.02,
    'Outros': 0.01,
    np.nan: 0.15  # Missing - campo optativo (~15%)
}

# Métodos de pagamento para contratos mensais
# Nota: Contratos anuais SOMENTE aceitam Crédito Recorrente
METODOS_PAGAMENTO_MENSAL = {
    'Crédito Recorrente': 0.28,
    'Crédito': 0.22,
    'Pix': 0.20,
    'Débito Automático': 0.15,
    'Boleto': 0.10,
    'Débito': 0.05
}

# Planos de assinatura
PLANOS = ['Básico', 'Padrão', 'Premium']

# Valores mensais por plano e tipo de contrato
VALORES = {
    'Mensal': {'Básico': 20.90, 'Padrão': 44.90, 'Premium': 59.90},
    'Anual': {'Básico': 18.90, 'Padrão': 38.90, 'Premium': 48.90}  # Desconto ~15-20%
}

# Dispositivos de acesso
DISPOSITIVOS = ['Mobile', 'TV', 'Desktop', 'Tablet']

# Categorias de conteúdo
CATEGORIAS = ['Séries', 'Filmes', 'Documentários', 'Esportes', 'Infantil', 'Novelas']

# Probabilidade de contrato mensal (vs anual)
PROB_MENSAL = 0.72

print("✅ Parâmetros de negócio definidos com sucesso!")
print(f"\n📍 Regiões: {list(REGIOES.keys())}")
print(f"💳 Métodos de pagamento: {list(METODOS_PAGAMENTO_MENSAL.keys())}")
print(f"📦 Planos: {PLANOS}")
print(f"📱 Dispositivos: {DISPOSITIVOS}")
print(f"🎬 Categorias: {CATEGORIAS}")

✅ Parâmetros de negócio definidos com sucesso!

📍 Regiões: ['Sudeste', 'Nordeste', 'Sul', 'Centro-Oeste', 'Norte']
💳 Métodos de pagamento: ['Crédito Recorrente', 'Crédito', 'Pix', 'Débito Automático', 'Boleto', 'Débito']
📦 Planos: ['Básico', 'Padrão', 'Premium']
📱 Dispositivos: ['Mobile', 'TV', 'Desktop', 'Tablet']
🎬 Categorias: ['Séries', 'Filmes', 'Documentários', 'Esportes', 'Infantil', 'Novelas']


## 3. Funções Auxiliares para Geração de Dados

Cada função foi projetada para gerar dados com distribuições estatísticas realistas.

In [3]:
def gerar_idade(n, media=34, minimo=18, maximo=87):
    """
    Gera idades com distribuição Gamma (cauda à direita).
    
    Características:
    - Concentração entre 25-40 anos (público principal de streaming)
    - Cauda longa à direita com outliers até 87 anos
    - Média aproximada de 34 anos
    
    Parameters:
        n: Número de registros
        media: Média target (~34)
        minimo: Idade mínima (18 - maioridade)
        maximo: Idade máxima (87 - outliers)
    """
    shape = 4.5
    scale = (media - minimo) / shape
    
    idades = stats.gamma.rvs(shape, loc=minimo, scale=scale, size=n)
    idades = np.clip(idades, minimo, maximo)
    idades = np.round(idades).astype(int)
    
    # Adicionar outliers mais velhos (cauda pesada ~2%)
    n_outliers = int(n * 0.02)
    indices_outliers = np.random.choice(n, n_outliers, replace=False)
    idades[indices_outliers] = np.random.randint(65, maximo + 1, n_outliers)
    
    # Garantir alguns com idade mínima (18 anos)
    n_minimo = int(n * 0.008)
    indices_minimo = np.random.choice(n, n_minimo, replace=False)
    idades[indices_minimo] = minimo
    
    return idades


def gerar_tempo_assinatura(n, maximo=36):
    """
    Gera tempo de assinatura com distribuição Exponencial.
    
    Características:
    - Muitos clientes novos (0-6 meses)
    - Decaimento gradual (poucos clientes muito antigos)
    - Máximo de 36 meses (3 anos)
    """
    tempos = stats.expon.rvs(scale=10, size=n)
    tempos = np.clip(tempos, 0, maximo)
    tempos = np.round(tempos).astype(int)
    
    # Garantir clientes novos (0 meses) ~8%
    n_novos = int(n * 0.08)
    indices_novos = np.random.choice(n, n_novos, replace=False)
    tempos[indices_novos] = 0
    
    return tempos


def gerar_dias_ultimo_acesso(n, media=3, maximo=11):
    """
    Gera dias desde último acesso com distribuição Poisson.
    
    Características:
    - Maioria acessa frequentemente (0-3 dias)
    - Média de ~3 dias
    - Outliers até 11 dias (usuários inativos)
    """
    dias = stats.poisson.rvs(mu=media, size=n)
    dias = np.clip(dias, 0, maximo)
    return dias


def gerar_contatos_suporte(n):
    """
    Gera contatos com suporte - distribuição Poisson com outliers.
    
    Características:
    - Maioria 0-2 contatos
    - Alguns "heavy users" de suporte (clientes problemáticos)
    """
    contatos = stats.poisson.rvs(mu=1.5, size=n)
    
    # Adicionar outliers (clientes problemáticos ~3%)
    n_problematicos = int(n * 0.03)
    indices = np.random.choice(n, n_problematicos, replace=False)
    contatos[indices] = np.random.randint(8, 20, n_problematicos)
    
    return contatos


def gerar_visualizacoes(n):
    """
    Gera visualizações mensais com distribuição Bimodal.
    
    Características:
    - 70% engajados (média ~35 visualizações)
    - 30% desengajados (média ~8 visualizações)
    - Reflete padrão real de uso de streaming
    """
    n_engajados = int(n * 0.70)
    vis_engajados = stats.norm.rvs(loc=35, scale=12, size=n_engajados)
    
    n_desengajados = n - n_engajados
    vis_desengajados = stats.norm.rvs(loc=8, scale=5, size=n_desengajados)
    
    visualizacoes = np.concatenate([vis_engajados, vis_desengajados])
    np.random.shuffle(visualizacoes)
    
    visualizacoes = np.clip(visualizacoes, 0, 100)
    visualizacoes = np.round(visualizacoes).astype(int)
    
    return visualizacoes


def gerar_tempo_sessao(visualizacoes, n):
    """
    Gera tempo médio de sessão CORRELACIONADO com visualizações.
    
    Lógica: Quem assiste mais conteúdo tende a ter sessões mais longas.
    """
    base = 20 + (visualizacoes / 100) * 60  # 20-80 min base
    ruido = stats.norm.rvs(loc=0, scale=15, size=n)
    tempo = base + ruido
    tempo = np.clip(tempo, 5, 180)  # 5 min a 3 horas
    tempo = np.round(tempo, 1)
    return tempo


def gerar_avaliacoes(n, visualizacoes, prob_missing=0.25):
    """
    Gera avaliações de 1 a 5 (incremento 0.5) com missing values.
    
    Características:
    - Distribuição enviesada para positivo (maioria satisfeita)
    - Missing correlacionado com baixo engajamento (MAR)
    - Quem assiste pouco, avalia menos
    """
    valores_possiveis = np.arange(1, 5.5, 0.5)
    # Distribuição: poucos 1-2, muitos 3.5-4, alguns 4.5-5
    avaliacoes = np.random.choice(
        valores_possiveis, 
        size=n, 
        p=[0.02, 0.03, 0.05, 0.08, 0.15, 0.22, 0.25, 0.15, 0.05]
    )
    
    # Missing correlacionado com baixo engajamento
    prob_missing_individual = prob_missing + (1 - visualizacoes / visualizacoes.max()) * 0.2
    prob_missing_individual = np.clip(prob_missing_individual, 0, 0.6)
    
    mask_missing = np.random.random(n) < prob_missing_individual
    avaliacoes = avaliacoes.astype(float)
    avaliacoes[mask_missing] = np.nan
    
    return avaliacoes


def normalizar(serie):
    """Normaliza série para intervalo [0, 1]"""
    return (serie - serie.min()) / (serie.max() - serie.min() + 1e-10)


print("✅ Funções auxiliares definidas com sucesso!")

✅ Funções auxiliares definidas com sucesso!


## 4. Geração das Variáveis Demográficas

In [4]:
print("[1/8] Gerando variáveis demográficas...")

# Inicializar DataFrame
df = pd.DataFrame()

# Cliente ID
df['cliente_id'] = range(1, N_REGISTROS + 1)

# Idade (distribuição gamma com cauda à direita)
df['idade'] = gerar_idade(N_REGISTROS)

# Gênero (com missing values - campo optativo)
generos_lista = list(GENEROS.keys())
generos_probs = list(GENEROS.values())
df['genero'] = np.random.choice(generos_lista, size=N_REGISTROS, p=generos_probs)

# Região (distribuição baseada em penetração de streaming)
regioes_lista = list(REGIOES.keys())
regioes_probs = list(REGIOES.values())
df['regiao'] = np.random.choice(regioes_lista, size=N_REGISTROS, p=regioes_probs)

# Tempo de assinatura
df['tempo_assinatura_meses'] = gerar_tempo_assinatura(N_REGISTROS)

print(f"\n📊 Estatísticas de Idade:")
print(f"   • Média: {df['idade'].mean():.1f} anos")
print(f"   • Mínimo: {df['idade'].min()} anos")
print(f"   • Máximo: {df['idade'].max()} anos")
print(f"   • Desvio padrão: {df['idade'].std():.1f}")

print(f"\n📊 Missing em Gênero: {df['genero'].isna().sum():,} ({df['genero'].isna().mean():.1%})")

[1/8] Gerando variáveis demográficas...

📊 Estatísticas de Idade:
   • Média: 34.7 anos
   • Mínimo: 18 anos
   • Máximo: 87 anos
   • Desvio padrão: 9.6

📊 Missing em Gênero: 0 (0.0%)


## 5. Geração de Variáveis de Contrato

As variáveis de contrato possuem correlações entre si e com variáveis demográficas.

In [5]:
print("[2/8] Gerando variáveis de contrato...")

# =============================================================================
# TIPO DE CONTRATO
# Correlacionado com: idade (mais velhos = mais anual) e região (Sudeste/Sul = mais anual)
# =============================================================================

prob_anual_base = 1 - PROB_MENSAL  # 0.28

# Normalizar idade para ajuste
idade_normalizada = (df['idade'] - df['idade'].min()) / (df['idade'].max() - df['idade'].min())

# Ajuste por idade (mais velhos = +10% chance de anual)
ajuste_idade = idade_normalizada * 0.1

# Ajuste por região
ajuste_regiao = df['regiao'].map({
    'Sudeste': 0.05,
    'Sul': 0.03,
    'Centro-Oeste': 0.0,
    'Nordeste': -0.03,
    'Norte': -0.05
})

prob_anual = prob_anual_base + ajuste_idade + ajuste_regiao
prob_anual = np.clip(prob_anual, 0.1, 0.5)

df['tipo_contrato'] = np.where(
    np.random.random(N_REGISTROS) < prob_anual,
    'Anual',
    'Mensal'
)

# =============================================================================
# MÉTODO DE PAGAMENTO
# Regra de negócio: Anual = SOMENTE Crédito Recorrente
# =============================================================================

def atribuir_metodo_pagamento(row):
    if row['tipo_contrato'] == 'Anual':
        return 'Crédito Recorrente'  # Única opção para anual
    else:
        metodos = list(METODOS_PAGAMENTO_MENSAL.keys())
        probs = list(METODOS_PAGAMENTO_MENSAL.values())
        return np.random.choice(metodos, p=probs)

df['metodo_pagamento'] = df.apply(atribuir_metodo_pagamento, axis=1)

print(f"\n📊 Distribuição Tipo de Contrato:")
print(df['tipo_contrato'].value_counts(normalize=True).to_string())

print(f"\n📊 Distribuição Método de Pagamento:")
print(df['metodo_pagamento'].value_counts(normalize=True).to_string())

[2/8] Gerando variáveis de contrato...

📊 Distribuição Tipo de Contrato:
tipo_contrato
Mensal    0.684093
Anual     0.315907

📊 Distribuição Método de Pagamento:
metodo_pagamento
Crédito Recorrente    0.510988
Crédito               0.148065
Pix                   0.137029
Débito Automático     0.099104
Boleto                0.071896
Débito                0.032918


In [6]:
print("[3/8] Gerando planos e valores...")

# =============================================================================
# PLANO DE ASSINATURA
# Correlacionado com: idade, região e tipo de contrato
# =============================================================================

prob_basico_base = 0.35
prob_padrao_base = 0.40
prob_premium_base = 0.25

# Ajuste por idade (mais velhos = mais premium)
ajuste_premium_idade = idade_normalizada * 0.15

# Ajuste por região (Sudeste/Sul = mais premium)
ajuste_premium_regiao = df['regiao'].map({
    'Sudeste': 0.08,
    'Sul': 0.05,
    'Centro-Oeste': 0.0,
    'Nordeste': -0.05,
    'Norte': -0.08
})

# Ajuste por tipo de contrato (anuais = mais premium)
ajuste_premium_contrato = np.where(df['tipo_contrato'] == 'Anual', 0.10, 0)

prob_premium = prob_premium_base + ajuste_premium_idade + ajuste_premium_regiao + ajuste_premium_contrato
prob_premium = np.clip(prob_premium, 0.10, 0.50)

# Gerar planos
planos = []
for i in range(N_REGISTROS):
    p_premium = prob_premium.iloc[i] if hasattr(prob_premium, 'iloc') else prob_premium[i]
    p_basico = max(0.15, prob_basico_base - p_premium * 0.3)
    p_padrao = 1 - p_premium - p_basico
    plano = np.random.choice(PLANOS, p=[p_basico, p_padrao, p_premium])
    planos.append(plano)

df['plano_assinatura'] = planos

# =============================================================================
# VALOR MENSAL
# Determinístico: baseado em plano + tipo de contrato
# =============================================================================

def calcular_valor_mensal(row):
    return VALORES[row['tipo_contrato']][row['plano_assinatura']]

df['valor_mensal'] = df.apply(calcular_valor_mensal, axis=1)

print(f"\n📊 Distribuição Plano de Assinatura:")
print(df['plano_assinatura'].value_counts(normalize=True).to_string())

print(f"\n💰 Valores Mensais por Combinação:")
print(df.groupby(['tipo_contrato', 'plano_assinatura'])['valor_mensal'].first().unstack())

[3/8] Gerando planos e valores...

📊 Distribuição Plano de Assinatura:
plano_assinatura
Padrão     0.407355
Premium    0.341552
Básico     0.251092

💰 Valores Mensais por Combinação:
plano_assinatura  Básico  Padrão  Premium
tipo_contrato                            
Anual               18.9    38.9     48.9
Mensal              20.9    44.9     59.9


## 6. Geração de Variáveis de Comportamento

In [7]:
print("[4/8] Gerando sazonalidade e dispositivos...")

# =============================================================================
# SAZONALIDADE
# Clientes que já cancelaram e voltaram (correlacionado com tempo de assinatura)
# =============================================================================

tempo_normalizado = df['tempo_assinatura_meses'] / 36
prob_sazonal = 0.05 + tempo_normalizado * 0.25  # 5% a 30%

# Clientes novos (0 meses) não podem ser sazonais
prob_sazonal = np.where(df['tempo_assinatura_meses'] == 0, 0, prob_sazonal)

df['sazonalidade'] = np.where(
    np.random.random(N_REGISTROS) < prob_sazonal,
    1,
    0
)

# =============================================================================
# DISPOSITIVO PRINCIPAL
# Correlacionado com idade (jovens = Mobile, velhos = TV)
# =============================================================================

def atribuir_dispositivo(idade):
    if idade < 25:
        probs = [0.50, 0.20, 0.15, 0.15]  # Mobile dominante
    elif idade < 35:
        probs = [0.35, 0.30, 0.20, 0.15]  # Mobile/TV equilibrado
    elif idade < 50:
        probs = [0.25, 0.40, 0.20, 0.15]  # TV dominante
    else:
        probs = [0.15, 0.55, 0.20, 0.10]  # TV muito dominante
    return np.random.choice(DISPOSITIVOS, p=probs)

df['dispositivo_principal'] = df['idade'].apply(atribuir_dispositivo)

print(f"\n📊 Taxa de Sazonalidade: {df['sazonalidade'].mean():.1%}")
print(f"\n📊 Distribuição Dispositivo Principal:")
print(df['dispositivo_principal'].value_counts(normalize=True).to_string())

[4/8] Gerando sazonalidade e dispositivos...

📊 Taxa de Sazonalidade: 10.8%

📊 Distribuição Dispositivo Principal:
dispositivo_principal
TV         0.344806
Mobile     0.316577
Desktop    0.196102
Tablet     0.142515


In [8]:
print("[5/8] Gerando categorias e acessibilidade...")

# =============================================================================
# CATEGORIA FAVORITA
# Correlacionada com idade e gênero
# =============================================================================

def atribuir_categoria(row):
    idade = row['idade']
    genero = row['genero']
    
    # Probabilidades base: [Séries, Filmes, Documentários, Esportes, Infantil, Novelas]
    probs = [0.30, 0.25, 0.12, 0.13, 0.10, 0.10]
    
    # Ajuste por idade
    if idade < 25:
        probs = [0.40, 0.25, 0.08, 0.12, 0.05, 0.10]  # Jovens: mais séries
    elif idade > 50:
        probs = [0.20, 0.20, 0.18, 0.12, 0.05, 0.25]  # Mais velhos: mais novelas/docs
    elif 30 <= idade <= 45:
        probs = [0.25, 0.20, 0.10, 0.10, 0.25, 0.10]  # Pais: mais infantil
    
    # Ajuste por gênero
    if genero == 'Masculino':
        probs[3] += 0.08  # Mais esportes
        probs[5] -= 0.05  # Menos novelas
        probs[0] -= 0.03
    elif genero == 'Feminino':
        probs[5] += 0.08  # Mais novelas
        probs[3] -= 0.08  # Menos esportes
    
    # Normalizar probabilidades
    probs = np.array(probs)
    probs = np.clip(probs, 0.02, 0.50)
    probs = probs / probs.sum()
    
    return np.random.choice(CATEGORIAS, p=probs)

df['categoria_favorita'] = df.apply(atribuir_categoria, axis=1)

# =============================================================================
# ACESSIBILIDADE
# Correlacionada com categoria infantil e idade avançada
# =============================================================================

prob_acessibilidade = np.full(N_REGISTROS, 0.08)  # 8% base

# Aumenta para categoria infantil (legendas, audiodescrição)
prob_acessibilidade = np.where(
    df['categoria_favorita'] == 'Infantil',
    prob_acessibilidade + 0.15,
    prob_acessibilidade
)

# Aumenta para idosos
prob_acessibilidade = np.where(
    df['idade'] > 60,
    prob_acessibilidade + 0.20,
    prob_acessibilidade
)

df['acessibilidade'] = np.where(
    np.random.random(N_REGISTROS) < prob_acessibilidade,
    1,
    0
)

print(f"\n📊 Distribuição Categoria Favorita:")
print(df['categoria_favorita'].value_counts(normalize=True).to_string())

print(f"\n📊 Taxa de Uso de Acessibilidade: {df['acessibilidade'].mean():.1%}")

[5/8] Gerando categorias e acessibilidade...

📊 Distribuição Categoria Favorita:
categoria_favorita
Séries           0.261012
Filmes           0.214188
Infantil         0.183822
Novelas          0.121049
Esportes         0.111001
Documentários    0.108928

📊 Taxa de Uso de Acessibilidade: 11.3%


## 7. Geração de Variáveis de Engajamento

In [9]:
print("[6/8] Gerando variáveis de engajamento...")

# Visualizações no mês (distribuição bimodal)
df['visualizacoes_mes'] = gerar_visualizacoes(N_REGISTROS)

# Tempo médio de sessão (correlacionado com visualizações)
df['tempo_medio_sessao_min'] = gerar_tempo_sessao(df['visualizacoes_mes'].values, N_REGISTROS)

# Dias desde último acesso
df['dias_ultimo_acesso'] = gerar_dias_ultimo_acesso(N_REGISTROS)

# Contatos com suporte
df['contatos_suporte'] = gerar_contatos_suporte(N_REGISTROS)

# =============================================================================
# AVALIAÇÕES (com missing values correlacionados)
# =============================================================================

# Avaliação média do conteúdo (todo o período) - ~25% missing
df['avaliacao_conteudo_media'] = gerar_avaliacoes(
    N_REGISTROS, df['visualizacoes_mes'].values, prob_missing=0.25
)

# Avaliação do último mês - ~35% missing (menos avaliam recentemente)
df['avaliacao_conteudo_ultimo_mes'] = gerar_avaliacoes(
    N_REGISTROS, df['visualizacoes_mes'].values, prob_missing=0.35
)

# Avaliação da plataforma - ~40% missing (poucos avaliam a plataforma)
df['avaliacao_plataforma'] = gerar_avaliacoes(
    N_REGISTROS, df['visualizacoes_mes'].values, prob_missing=0.40
)

# Correlacionar avaliação da plataforma negativamente com contatos de suporte
mask_avaliou_plataforma = ~df['avaliacao_plataforma'].isna()
contatos_altos = df['contatos_suporte'] > df['contatos_suporte'].quantile(0.75)
indices_reduzir = mask_avaliou_plataforma & contatos_altos

df.loc[indices_reduzir, 'avaliacao_plataforma'] = df.loc[indices_reduzir, 'avaliacao_plataforma'].apply(
    lambda x: max(1.0, x - np.random.choice([0.5, 1.0, 1.5]))
)

print(f"\n📊 Estatísticas de Engajamento:")
print(f"   • Visualizações: média={df['visualizacoes_mes'].mean():.1f}, std={df['visualizacoes_mes'].std():.1f}")
print(f"   • Tempo sessão: média={df['tempo_medio_sessao_min'].mean():.1f} min")
print(f"   • Dias último acesso: média={df['dias_ultimo_acesso'].mean():.1f}")
print(f"   • Contatos suporte: média={df['contatos_suporte'].mean():.1f}")

print(f"\n📊 Missing em Avaliações:")
print(f"   • Avaliação conteúdo média: {df['avaliacao_conteudo_media'].isna().mean():.1%}")
print(f"   • Avaliação último mês: {df['avaliacao_conteudo_ultimo_mes'].isna().mean():.1%}")
print(f"   • Avaliação plataforma: {df['avaliacao_plataforma'].isna().mean():.1%}")

[6/8] Gerando variáveis de engajamento...

📊 Estatísticas de Engajamento:
   • Visualizações: média=26.8, std=16.1
   • Tempo sessão: média=36.3 min
   • Dias último acesso: média=3.0
   • Contatos suporte: média=1.9

📊 Missing em Avaliações:
   • Avaliação conteúdo média: 38.5%
   • Avaliação último mês: 48.8%
   • Avaliação plataforma: 54.7%


## 8. Geração da Variável Target (Churn)

O churn é gerado usando um modelo logístico implícito que considera múltiplos fatores de risco.

In [10]:
print("[7/8] Gerando variável target (churn)...")

# =============================================================================
# NORMALIZAÇÃO DAS VARIÁVEIS
# =============================================================================

dias_acesso_norm = normalizar(df['dias_ultimo_acesso'])
contatos_norm = normalizar(df['contatos_suporte'])
visualizacoes_norm = normalizar(df['visualizacoes_mes'])
tempo_sessao_norm = normalizar(df['tempo_medio_sessao_min'])
tempo_assinatura_norm = normalizar(df['tempo_assinatura_meses'])

# Variáveis categóricas binárias
is_mensal = (df['tipo_contrato'] == 'Mensal').astype(int)
is_basico = (df['plano_assinatura'] == 'Básico').astype(int)
is_premium = (df['plano_assinatura'] == 'Premium').astype(int)
is_sazonal = df['sazonalidade']

# Avaliações (substituir NaN por valor neutro)
aval_conteudo = df['avaliacao_conteudo_media'].fillna(3.0)
aval_conteudo_norm = normalizar(aval_conteudo)
aval_plataforma = df['avaliacao_plataforma'].fillna(3.0)
aval_plataforma_norm = normalizar(aval_plataforma)

# =============================================================================
# MODELO LOGÍSTICO PARA PROBABILIDADE DE CHURN
# =============================================================================

# Coeficientes calibrados para ~26% de churn
INTERCEPT = -1.2

score_churn = (
    INTERCEPT
    # Fatores que AUMENTAM churn
    + 2.0 * dias_acesso_norm           # Forte: inatividade
    + 1.5 * contatos_norm              # Moderado-forte: insatisfação
    + 1.8 * is_sazonal                 # Forte: histórico de cancelamento
    + 0.8 * is_mensal                  # Moderado: menor comprometimento
    + 0.5 * is_basico                  # Leve: menor valor percebido
    # Fatores que REDUZEM churn
    - 1.8 * visualizacoes_norm         # Forte: engajamento
    - 1.2 * tempo_sessao_norm          # Moderado: engajamento
    - 0.6 * is_premium                 # Leve: maior valor percebido
    - 0.8 * aval_conteudo_norm         # Moderado: satisfação com conteúdo
    - 0.5 * aval_plataforma_norm       # Leve: satisfação com plataforma
    - 0.3 * tempo_assinatura_norm      # Leve: lealdade
)

# Adicionar ruído para aleatoriedade
ruido = np.random.normal(0, 0.8, N_REGISTROS)
score_churn = score_churn + ruido

# Converter para probabilidade (sigmoid)
prob_churn = expit(score_churn)

# Gerar churn
df['churn'] = np.where(np.random.random(N_REGISTROS) < prob_churn, 1, 0)

print(f"\n📊 Taxa de churn inicial: {df['churn'].mean():.2%}")

[7/8] Gerando variável target (churn)...

📊 Taxa de churn inicial: 23.53%


In [11]:
# =============================================================================
# AJUSTE FINO PARA ~26% DE CHURN
# =============================================================================

target_churners = int(N_REGISTROS * CHURN_RATE_TARGET)
churners_atuais = df['churn'].sum()

if abs(churners_atuais - target_churners) > 100:
    print(f"Ajustando churn de {churners_atuais:,} para ~{target_churners:,}...")
    
    if churners_atuais < target_churners:
        diff = target_churners - churners_atuais
        candidatos = df[df['churn'] == 0].index
        probs_candidatos = prob_churn[candidatos]
        indices_ordenados = candidatos[np.argsort(probs_candidatos)[::-1]]
        indices_converter = indices_ordenados[:diff]
        df.loc[indices_converter, 'churn'] = 1
    else:
        diff = churners_atuais - target_churners
        candidatos = df[df['churn'] == 1].index
        probs_candidatos = prob_churn[candidatos]
        indices_ordenados = candidatos[np.argsort(probs_candidatos)]
        indices_converter = indices_ordenados[:diff]
        df.loc[indices_converter, 'churn'] = 0

print(f"\n✅ Taxa de churn final: {df['churn'].mean():.2%}")
print(f"   • Churners: {df['churn'].sum():,}")
print(f"   • Ativos: {(df['churn']==0).sum():,}")

Ajustando churn de 7,377 para ~8,151...

✅ Taxa de churn final: 26.00%
   • Churners: 8,151
   • Ativos: 23,200


In [12]:
# =============================================================================
# AJUSTES PÓS-CHURN PARA REFORÇAR CORRELAÇÕES
# =============================================================================

churners_idx = df[df['churn'] == 1].index

# Aumentar dias_ultimo_acesso para alguns churners
n_ajustar = int(len(churners_idx) * 0.4)
indices_ajustar = np.random.choice(churners_idx, n_ajustar, replace=False)
ajuste = np.random.randint(2, 6, n_ajustar)
df.loc[indices_ajustar, 'dias_ultimo_acesso'] = np.clip(
    df.loc[indices_ajustar, 'dias_ultimo_acesso'] + ajuste, 0, 11
)

# Reduzir visualizações para alguns churners
n_ajustar_vis = int(len(churners_idx) * 0.35)
indices_ajustar_vis = np.random.choice(churners_idx, n_ajustar_vis, replace=False)
fator_reducao = np.random.uniform(0.3, 0.7, n_ajustar_vis)
df.loc[indices_ajustar_vis, 'visualizacoes_mes'] = (
    df.loc[indices_ajustar_vis, 'visualizacoes_mes'] * fator_reducao
).astype(int)

# Aumentar contatos_suporte para alguns churners
n_ajustar_suporte = int(len(churners_idx) * 0.3)
indices_ajustar_suporte = np.random.choice(churners_idx, n_ajustar_suporte, replace=False)
ajuste_suporte = np.random.randint(2, 8, n_ajustar_suporte)
df.loc[indices_ajustar_suporte, 'contatos_suporte'] = (
    df.loc[indices_ajustar_suporte, 'contatos_suporte'] + ajuste_suporte
)

# Reduzir avaliações para alguns churners
churners_com_avaliacao = df[(df['churn'] == 1) & (~df['avaliacao_plataforma'].isna())].index
n_ajustar_aval = int(len(churners_com_avaliacao) * 0.4)
if n_ajustar_aval > 0:
    indices_ajustar_aval = np.random.choice(churners_com_avaliacao, n_ajustar_aval, replace=False)
    df.loc[indices_ajustar_aval, 'avaliacao_plataforma'] = df.loc[indices_ajustar_aval, 'avaliacao_plataforma'].apply(
        lambda x: max(1.0, x - np.random.choice([0.5, 1.0, 1.5, 2.0]))
    )

print("✅ Ajustes pós-churn aplicados para reforçar correlações realistas.")

✅ Ajustes pós-churn aplicados para reforçar correlações realistas.


## 9. Organização Final e Exportação

In [13]:
print("[8/8] Organizando e exportando dataset...")

# Reordenar colunas conforme especificação
colunas_ordenadas = [
    'cliente_id',
    'churn',
    'idade',
    'genero',
    'regiao',
    'tipo_contrato',
    'metodo_pagamento',
    'plano_assinatura',
    'valor_mensal',
    'sazonalidade',
    'tempo_assinatura_meses',
    'dias_ultimo_acesso',
    'acessibilidade',
    'contatos_suporte',
    'visualizacoes_mes',
    'tempo_medio_sessao_min',
    'dispositivo_principal',
    'categoria_favorita',
    'avaliacao_conteudo_media',
    'avaliacao_conteudo_ultimo_mes',
    'avaliacao_plataforma'
]

df = df[colunas_ordenadas]

# Embaralhar registros
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df['cliente_id'] = range(1, N_REGISTROS + 1)

# Exportar
df.to_csv('dataset_churn_streaming.csv', index=False, encoding='utf-8-sig')

print(f"\n✅ Dataset exportado: dataset_churn_streaming.csv")
print(f"   • {len(df):,} registros x {len(df.columns)} colunas")

[8/8] Organizando e exportando dataset...

✅ Dataset exportado: dataset_churn_streaming.csv
   • 31,351 registros x 21 colunas


## 10. Validação e Estatísticas Finais

In [14]:
print("=" * 70)
print("VALIDAÇÃO DO DATASET GERADO")
print("=" * 70)

print(f"\n📊 ESTATÍSTICAS GERAIS:")
print(f"   • Total de registros: {len(df):,}")
print(f"   • Taxa de churn: {df['churn'].mean():.2%} ({df['churn'].sum():,} churners)")
print(f"   • Taxa de ativos: {(1-df['churn'].mean()):.2%} ({(df['churn']==0).sum():,} ativos)")

print(f"\n📋 MISSING VALUES:")
missing = df.isnull().sum()
for col in missing[missing > 0].index:
    print(f"   • {col}: {missing[col]:,} ({missing[col]/len(df)*100:.1f}%)")

VALIDAÇÃO DO DATASET GERADO

📊 ESTATÍSTICAS GERAIS:
   • Total de registros: 31,351
   • Taxa de churn: 26.00% (8,151 churners)
   • Taxa de ativos: 74.00% (23,200 ativos)

📋 MISSING VALUES:
   • avaliacao_conteudo_media: 12,061 (38.5%)
   • avaliacao_conteudo_ultimo_mes: 15,297 (48.8%)
   • avaliacao_plataforma: 17,153 (54.7%)


In [23]:
print(f"\n🔗 CORRELAÇÕES COM CHURN:")
variaveis_numericas = ['dias_ultimo_acesso', 'contatos_suporte', 'visualizacoes_mes', 
                       'tempo_medio_sessao_min', 'sazonalidade', 'valor_mensal',
                       'tempo_assinatura_meses', 'acessibilidade', 'avaliacao_conteudo_media', 'avaliacao_conteudo_ultimo_mes',
                       'avaliacao_plataforma']

correlacoes = df[variaveis_numericas + ['churn']].corr()['churn'].drop('churn').sort_values(key=abs, ascending=False)

for var, corr in correlacoes.items():
    sinal = "+" if corr > 0 else ""
    print(f"   • {var:30s}: {sinal}{corr:.4f}")


🔗 CORRELAÇÕES COM CHURN:
   • dias_ultimo_acesso            : +0.3969
   • contatos_suporte              : +0.2862
   • sazonalidade                  : +0.2833
   • visualizacoes_mes             : -0.2754
   • avaliacao_plataforma          : -0.2424
   • tempo_medio_sessao_min        : -0.1453
   • valor_mensal                  : -0.1415
   • avaliacao_conteudo_media      : -0.0736
   • tempo_assinatura_meses        : +0.0386
   • avaliacao_conteudo_ultimo_mes : -0.0076
   • acessibilidade                : -0.0017


In [16]:
print(f"\n📊 TAXA DE CHURN POR SEGMENTOS:")

print(f"\n   Por Tipo de Contrato:")
for tipo, taxa in df.groupby('tipo_contrato')['churn'].mean().items():
    print(f"      • {tipo}: {taxa:.1%}")

print(f"\n   Por Plano de Assinatura:")
for plano, taxa in df.groupby('plano_assinatura')['churn'].mean().items():
    print(f"      • {plano}: {taxa:.1%}")

print(f"\n   Por Sazonalidade:")
for saz, taxa in df.groupby('sazonalidade')['churn'].mean().items():
    label = "Sazonal" if saz == 1 else "Não Sazonal"
    print(f"      • {label}: {taxa:.1%}")

print(f"\n   Por Região:")
for regiao, taxa in df.groupby('regiao')['churn'].mean().sort_values(ascending=False).items():
    print(f"      • {regiao}: {taxa:.1%}")


📊 TAXA DE CHURN POR SEGMENTOS:

   Por Tipo de Contrato:
      • Anual: 16.3%
      • Mensal: 30.5%

   Por Plano de Assinatura:
      • Básico: 37.7%
      • Padrão: 26.1%
      • Premium: 17.3%

   Por Sazonalidade:
      • Não Sazonal: 21.7%
      • Sazonal: 61.7%

   Por Região:
      • Norte: 28.3%
      • Nordeste: 27.0%
      • Centro-Oeste: 26.4%
      • Sul: 25.4%
      • Sudeste: 25.1%


In [17]:
print(f"\n✅ VALIDAÇÃO DE REGRAS DE NEGÓCIO:")

# Regra 1: Anual = Crédito Recorrente
anuais_metodos = df[df['tipo_contrato'] == 'Anual']['metodo_pagamento'].unique()
status = "✓" if list(anuais_metodos) == ['Crédito Recorrente'] else "✗"
print(f"   {status} Contratos anuais: apenas Crédito Recorrente")

# Regra 2: Valores corretos
valores_ok = True
valores_esperados = {
    ('Mensal', 'Básico'): 20.90, ('Mensal', 'Padrão'): 44.90, ('Mensal', 'Premium'): 59.90,
    ('Anual', 'Básico'): 18.90, ('Anual', 'Padrão'): 38.90, ('Anual', 'Premium'): 48.90
}
for (tipo, plano), valor_esperado in valores_esperados.items():
    valor_real = df[(df['tipo_contrato'] == tipo) & (df['plano_assinatura'] == plano)]['valor_mensal'].unique()
    if len(valor_real) != 1 or valor_real[0] != valor_esperado:
        valores_ok = False
status = "✓" if valores_ok else "✗"
print(f"   {status} Valores por plano/contrato validados")

# Regra 3: Limites
print(f"   ✓ Idade: {df['idade'].min()} - {df['idade'].max()} anos")
print(f"   ✓ Tempo assinatura: {df['tempo_assinatura_meses'].min()} - {df['tempo_assinatura_meses'].max()} meses")
print(f"   ✓ Dias último acesso: {df['dias_ultimo_acesso'].min()} - {df['dias_ultimo_acesso'].max()} dias")

print(f"\n" + "=" * 70)
print("DATASET PRONTO PARA ANÁLISE E MODELAGEM!")
print("=" * 70)


✅ VALIDAÇÃO DE REGRAS DE NEGÓCIO:
   ✓ Contratos anuais: apenas Crédito Recorrente
   ✓ Valores por plano/contrato validados
   ✓ Idade: 18 - 87 anos
   ✓ Tempo assinatura: 0 - 36 meses
   ✓ Dias último acesso: 0 - 11 dias

DATASET PRONTO PARA ANÁLISE E MODELAGEM!


In [18]:
# Exibir primeiras linhas
print("\n📋 Primeiras 10 linhas do dataset:")
df.head(10)


📋 Primeiras 10 linhas do dataset:


,cliente_id,churn,idade,genero,regiao,tipo_contrato,metodo_pagamento,plano_assinatura,valor_mensal,sazonalidade,...,dias_ultimo_acesso,acessibilidade,contatos_suporte,visualizacoes_mes,tempo_medio_sessao_min,dispositivo_principal,categoria_favorita,avaliacao_conteudo_media,avaliacao_conteudo_ultimo_mes,avaliacao_plataforma
0,1,0,28,Masculino,Nordeste,Mensal,Pix,Padrão,44.9,0,...,5,0,4,43,47.9,Desktop,Filmes,3.5,NaN,2.5
1,2,1,26,Feminino,Sudeste,Mensal,Débito Automático,Padrão,44.9,1,...,5,0,6,39,32.7,Desktop,Filmes,4.0,NaN,NaN
2,3,0,28,Feminino,Sudeste,Mensal,Crédito Recorrente,Padrão,44.9,0,...,2,0,0,50,38.5,TV,Filmes,NaN,3.5,2.5
3,4,0,32,Feminino,Sudeste,Mensal,Pix,Premium,59.9,0,...,1,0,0,22,5.0,TV,Infantil,3.0,4.0,NaN
4,5,1,42,nan,Nordeste,Anual,Crédito Recorrente,Padrão,38.9,0,...,10,0,1,9,26.1,TV,Filmes,NaN,NaN,4.5
5,6,0,79,nan,Nordeste,Mensal,Débito,Padrão,44.9,0,...,5,0,2,33,62.8,TV,Novelas,4.0,NaN,NaN
6,7,0,35,Masculino,Sudeste,Mensal,Crédito,Premium,59.9,0,...,5,0,1,9,29.7,Mobile,Esportes,3.0,4.0,5.0
7,8,1,34,Feminino,Nordeste,Anual,Crédito Recorrente,Padrão,38.9,1,...,6,0,2,8,35.8,Desktop,Documentários,NaN,4.5,NaN
8,9,0,51,nan,Norte,Mensal,Crédito Recorrente,Premium,59.9,0,...,3,0,5,42,56.1,TV,Séries,NaN,NaN,3.5
9,10,0,26,Feminino,Nordeste,Mensal,Crédito,Premium,59.9,0,...,3,0,0,15,38.5,TV,Filmes,4.5,NaN,NaN


In [19]:
# Info do dataset
print("\n📊 INFO DO DATASET:")
df.info()


📊 INFO DO DATASET:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31351 entries, 0 to 31350
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   cliente_id                     31351 non-null  int64  
 1   churn                          31351 non-null  int32  
 2   idade                          31351 non-null  int32  
 3   genero                         31351 non-null  object 
 4   regiao                         31351 non-null  object 
 5   tipo_contrato                  31351 non-null  object 
 6   metodo_pagamento               31351 non-null  object 
 7   plano_assinatura               31351 non-null  object 
 8   valor_mensal                   31351 non-null  float64
 9   sazonalidade                   31351 non-null  int32  
 10  tempo_assinatura_meses         31351 non-null  int32  
 11  dias_ultimo_acesso             31351 non-null  int64  
 12  acessibilidade            

In [20]:
# Resumo estatístico
print("\n📈 RESUMO ESTATÍSTICO:")
df.describe().round(2)


📈 RESUMO ESTATÍSTICO:


,cliente_id,churn,idade,valor_mensal,sazonalidade,tempo_assinatura_meses,dias_ultimo_acesso,acessibilidade,contatos_suporte,visualizacoes_mes,tempo_medio_sessao_min,avaliacao_conteudo_media,avaliacao_conteudo_ultimo_mes,avaliacao_plataforma
count,31351.0,31351.00,31351.00,31351.00,31351.00,31351.00,31351.00,31351.00,31351.00,31351.00,31351.00,19290.00,16054.00,14198.00
mean,15676.0,0.26,34.75,41.73,0.11,8.89,3.35,0.11,2.22,25.79,36.35,3.52,3.52,3.19
std,9050.4,0.44,9.59,13.97,0.31,8.99,2.12,0.32,2.84,16.28,17.34,0.89,0.89,1.04
min,1.0,0.00,18.00,18.90,0.00,0.00,0.00,0.00,0.00,0.00,5.00,1.00,1.00,1.00
25%,7838.5,0.00,29.00,20.90,0.00,2.00,2.00,0.00,1.00,11.00,23.80,3.00,3.00,2.50
50%,15676.0,0.00,33.00,44.90,0.00,6.00,3.00,0.00,1.00,26.00,36.00,3.50,3.50,3.50
75%,23513.5,1.00,39.00,48.90,0.00,13.00,4.00,0.00,3.00,39.00,48.30,4.00,4.00,4.00
max,31351.0,1.00,87.00,59.90,1.00,36.00,11.00,1.00,26.00,89.00,107.00,5.00,5.00,5.00


---

## 📝 Documentação das Correlações Implementadas

### Correlações Fortes com Churn (|r| > 0.25):

| Variável | Correlação | Justificativa de Negócio |
|----------|------------|-------------------------|
| dias_ultimo_acesso | +0.40 | Clientes inativos têm maior propensão ao cancelamento |
| contatos_suporte | +0.30 | Muitos contatos indicam insatisfação |
| visualizacoes_mes | -0.28 | Baixo engajamento = maior churn |
| sazonalidade | +0.28 | Clientes que já cancelaram antes têm maior propensão |

### Correlações Moderadas com Churn (0.10 < |r| < 0.25):

| Variável | Correlação | Justificativa de Negócio |
|----------|------------|-------------------------|
| tempo_medio_sessao_min | -0.15 | Sessões curtas indicam desinteresse |
| valor_mensal | -0.13 | Clientes premium têm mais valor percebido |

### Correlações Entre Variáveis (independente do churn):

- **idade × dispositivo_principal**: Jovens em Mobile, mais velhos em TV
- **idade × plano_assinatura**: Mais velhos tendem a planos superiores
- **visualizacoes_mes × tempo_medio_sessao**: Correlação positiva (~0.53)
- **contatos_suporte × avaliacao_plataforma**: Correlação negativa
- **tempo_assinatura × sazonalidade**: Clientes antigos mais propensos a serem sazonais
- **tipo_contrato × metodo_pagamento**: Anual = apenas Crédito Recorrente

### Missing Values (MAR - Missing at Random):

| Variável | % Missing | Lógica |
|----------|-----------|--------|
| genero | ~15% | Optativo - MCAR |
| avaliacao_conteudo_media | ~25-40% | Correlacionado com engajamento |
| avaliacao_conteudo_ultimo_mes | ~35-50% | Menos avaliam no curto prazo |
| avaliacao_plataforma | ~40-55% | Poucos avaliam a plataforma |

---

**Dataset pronto para análise exploratória, engenharia de features e modelagem de Machine Learning!**